In [ ]:
using LinearAlgebra

eigenvalues = [0.4, 0, 1, 2]

diagonal_matrix = diagm(eigenvalues)

square_root_matrix = sqrt.(diagonal_matrix)

In [ ]:
# using LinearAlgebra
# using JLD2

# function load_eigenresults(output_file="eigen_results.jld2")
#     # Unpack directly into variables
#     data = load(output_file)  # Returns a Dict-like object
#     Σexact = data["Σexact"]  # Access by key
#     Uexact = data["Uexact"]
#     return Σexact, Uexact
# end

# Sigma, Uexact = load_eigenresults("../MA_best/Eigenvalues_folder/eigen_results_hBN.jld2")

In [ ]:
using LinearAlgebra
using IterativeSolvers

# ------------------------------------------------------------
# Helper function: project vector orthogonal to v
function project(v::AbstractVector, x::AbstractMatrix)
    return x - v * v' * x
end

# ------------------------------------------------------------
# Helper function: project matrix-vector product operator
function M_operator(A, v, λ)
    I_mat = I(size(A, 1))  # Identity matrix of the same size as A
    x = (A - λ * I_mat)
    y = project(v, x) 
    z = project(v, y)
    return z
end

A = rand(100, 100)  # Example matrix A

In [ ]:
nev = 5
n = 100
λs = ones(nev)
R = ones(nev)

V = zeros(n, nev)
for i = 1:nev
    V[i,i] = 1.0
end


v = V[:,1]
λ = λs[1]
r = R[1]

# Define LinearOperator M
Mop = M_operator(A, v, λ)    
display(Mop)

println(Mop * v)


In [8]:
using IterativeSolvers, BenchmarkTools, LinearAlgebra, SparseArrays

n = 1000
A = sprandn(n, n, 0.01)
A = A + A' + n*I # make it SPD for cg
b = randn(n)

println("Benchmarking CG")
@btime cg($A, $b; maxiter=100, reltol=1e-6)
println("Benchmarking MINRES")
@btime minres($A, $b; maxiter=100, reltol=1e-6)

Benchmarking CG
  51.860 μs (20 allocations: 32.14 KiB)
Benchmarking MINRES
  54.134 μs (33 allocations: 56.00 KiB)


1000-element Vector{Float64}:
  0.00014329594989212938
  0.0006670605998509154
  0.0006751435710649825
 -0.0006560861037841071
 -0.0027933714759489896
  0.0007129606558824044
  0.001057386016810772
  9.033353259657234e-5
  0.00022198555209894855
 -0.0005522052771966552
  ⋮
 -0.0026376529365174524
  0.001434576857738931
  0.001305169049233081
  0.000250735689868935
 -0.0010848815342644747
  0.0016136044377277317
 -0.0006065581550458612
  0.0012172872558652583
  0.00033706938298561334